**Executed:** Sat Mar 18 01:18:00 2017

**Duration:** 4 seconds.

**Autogenerated from:** [./param-learning-template.ipynb](./param-learning-template.ipynb)

In [1]:
# default values of file paths, assuming quantum_fog is working dir
in_bif = 'examples_cbnets/earthquake.bif'
in_dot = 'examples_cbnets/earthquake.dot'
in_csv = 'learning/training_data_c/earthquake.csv'
qfog_path = None

In [2]:
# Cell inserted during automated execution.
in_bif = 'examples_cbnets/asia_.bif'
in_dot = 'examples_cbnets/asia_.dot'
in_csv = 'learning/training_data_c/tempo.csv'
qfog_path = '/home/jupyter/Notebooks/Quantum/quantum-fog'

**Parameter Learning Template**

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Learned-Network-Parameters" data-toc-modified-id="Learned-Network-Parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learned Network Parameters</a></div><div class="lev2 toc-item"><a href="#QFog" data-toc-modified-id="QFog-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>QFog</a></div><div class="lev2 toc-item"><a href="#bnlearn" data-toc-modified-id="bnlearn-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>bnlearn</a></div><div class="lev2 toc-item"><a href="#Summary-of-Running-Times" data-toc-modified-id="Summary-of-Running-Times-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Summary of Running Times</a></div>

In [3]:
import pandas as pd
import numpy as np
from graphviz import Source

import warnings
warnings.filterwarnings("ignore", module="rpy2")

import rpy2
%load_ext rpy2.ipython
%R library("bnlearn");
%R library("Rgraphviz");

In [4]:
import os
import sys
if not qfog_path:
    os.chdir('../../')
else:
    os.chdir(qfog_path)
cwd = os.getcwd()
sys.path.insert(0,cwd)
print("cwd=", cwd)
from learning.NetParamsLner import *

cwd= /home/jupyter/Notebooks/Quantum/quantum-fog


Read in_csv, create Pandas Dataframe with it, push dataframe into R

In [5]:
states_df = pd.read_csv(in_csv)
states_df.head()

,Visited_Asian_TB_HRA,Smoke,Lung_Cancer,Bronchitis,Tuberculosis,Either,X_Ray,Dyspnea
0,1,0,1,0,1,1,1,0
1,1,0,1,0,1,1,1,0
2,1,1,1,1,1,1,1,1
3,1,0,1,1,1,1,1,1
4,1,0,1,1,1,1,1,1


In [6]:
states_df.tail()

,Visited_Asian_TB_HRA,Smoke,Lung_Cancer,Bronchitis,Tuberculosis,Either,X_Ray,Dyspnea
1995,1,1,1,1,1,1,1,1
1996,1,1,1,0,1,1,1,0
1997,1,0,0,0,1,0,0,0
1998,1,0,1,1,1,1,1,1
1999,1,1,1,0,1,1,1,0


In [7]:
%Rpush states_df
%R str(states_df)

'data.frame':	2000 obs. of  8 variables:
 $ Visited_Asian_TB_HRA: int  1 1 1 1 1 1 1 1 1 1 ...
 $ Smoke               : int  0 0 1 0 0 0 0 1 0 0 ...
 $ Lung_Cancer         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Bronchitis          : int  0 0 1 1 1 0 1 1 0 0 ...
 $ Tuberculosis        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Either              : int  1 1 1 1 1 1 1 1 1 1 ...
 $ X_Ray               : int  1 1 1 1 1 1 1 1 0 1 ...
 $ Dyspnea             : int  0 0 1 1 1 0 0 1 0 0 ...


For bnlearn, data.frame columns cannot be int type, must be changed to factor.
data.frame is a list so can use lapply

In [8]:
%R states_df[] <- lapply(states_df, factor)
%R str(states_df)

'data.frame':	2000 obs. of  8 variables:
 $ Visited_Asian_TB_HRA: Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Smoke               : Factor w/ 2 levels "0","1": 1 1 2 1 1 1 1 2 1 1 ...
 $ Lung_Cancer         : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Bronchitis          : Factor w/ 2 levels "0","1": 1 1 2 2 2 1 2 2 1 1 ...
 $ Tuberculosis        : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Either              : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ X_Ray               : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 1 2 ...
 $ Dyspnea             : Factor w/ 2 levels "0","1": 1 1 2 2 2 1 1 2 1 1 ...


# Learned Network Parameters

## QFog

In [9]:
# emp = empirical, learned
is_quantum = False
bnet = BayesNet.read_bif(in_bif, is_quantum)
bnet_emp = BayesNet.read_dot(in_dot)
vtx_to_states = bnet.get_vtx_to_state_names()
bnet_emp.import_nd_state_names(vtx_to_states)


lnr = NetParamsLner(is_quantum, bnet_emp, states_df)

In [10]:
%%capture qfog_params_time
%time lnr.learn_all_bnet_pots()

In [11]:
print(qfog_params_time)

CPU times: user 56.6 ms, sys: 30 us, total: 56.6 ms
Wall time: 56.9 ms



In [12]:
lnr.compare_true_and_emp_pots(bnet, bnet_emp)


node= Smoke
true:
['Smoke']
[ 0.5  0.5]
empirical:
['Smoke']
[ 0.499  0.501]

node= Visited_Asian_TB_HRA
true:
['Visited_Asian_TB_HRA']
[ 0.01  0.99]
empirical:
['Visited_Asian_TB_HRA']
[ 0.009  0.991]

node= X_Ray
true:
['Either', 'X_Ray']
[[ 0.98  0.02]
 [ 0.05  0.95]]
empirical:
['Either', 'X_Ray']
[[ 0.95522388  0.04477612]
 [ 0.04662379  0.95337621]]

node= Either
true:
['Lung_Cancer', 'Tuberculosis', 'Either']
[[[ 1.  0.]
  [ 1.  0.]]

 [[ 1.  0.]
  [ 0.  1.]]]
empirical:
['Lung_Cancer', 'Tuberculosis', 'Either']
[[[ 1.  0.]
  [ 1.  0.]]

 [[ 1.  0.]
  [ 0.  1.]]]

node= Bronchitis
true:
['Smoke', 'Bronchitis']
[[ 0.6  0.4]
 [ 0.3  0.7]]
empirical:
['Smoke', 'Bronchitis']
[[ 0.60721443  0.39278557]
 [ 0.30239521  0.69760479]]

node= Dyspnea
true:
['Bronchitis', 'Either', 'Dyspnea']
[[[ 0.9  0.1]
  [ 0.7  0.3]]

 [[ 0.8  0.2]
  [ 0.1  0.9]]]
empirical:
['Bronchitis', 'Either', 'Dyspnea']
[[[ 0.90277778  0.09722222]
  [ 0.70011947  0.29988053]]

 [[ 0.88709677  0.11290323]
  [ 0.0

## bnlearn


In [13]:
%Rpush in_bif
%R bn.fit = read.bif(in_bif)
%R str(bn.fit)

List of 8
 $ Visited_Asian_TB_HRA:List of 4
  ..$ node    : chr "Visited_Asian_TB_HRA"
  ..$ parents : chr(0) 
  ..$ children: chr "Tuberculosis"
  ..$ prob    : table [1:2(1d)] 0.01 0.99
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "yes" "no"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Tuberculosis        :List of 4
  ..$ node    : chr "Tuberculosis"
  ..$ parents : chr "Visited_Asian_TB_HRA"
  ..$ children: chr "Either"
  ..$ prob    : table [1:2, 1:2] 0.05 0.95 0.01 0.99
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ Tuberculosis        : chr [1:2] "yes" "no"
  .. .. ..$ Visited_Asian_TB_HRA: chr [1:2] "yes" "no"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Smoke               :List of 4
  ..$ node    : chr "Smoke"
  ..$ parents : chr(0) 
  ..$ children: chr [1:2] "Lung_Cancer" "Bronchitis"
  ..$ prob    : table [1:2(1d)] 0.5 0.5
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "yes" "no"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Lung_Cance

In [14]:
%R bn = bn.net(bn.fit)
%R str(bn)

List of 3
 $ learning:List of 6
  ..$ whitelist: NULL
  ..$ blacklist: NULL
  ..$ test     : chr "none"
  ..$ ntests   : num 0
  ..$ algo     : chr "empty"
  ..$ args     : list()
 $ nodes   :List of 8
  ..$ Visited_Asian_TB_HRA:List of 4
  .. ..$ mb      : chr "Tuberculosis"
  .. ..$ nbr     : chr "Tuberculosis"
  .. ..$ parents : chr(0) 
  .. ..$ children: chr "Tuberculosis"
  ..$ Tuberculosis        :List of 4
  .. ..$ mb      : chr [1:3] "Visited_Asian_TB_HRA" "Lung_Cancer" "Either"
  .. ..$ nbr     : chr [1:2] "Visited_Asian_TB_HRA" "Either"
  .. ..$ parents : chr "Visited_Asian_TB_HRA"
  .. ..$ children: chr "Either"
  ..$ Smoke               :List of 4
  .. ..$ mb      : chr [1:2] "Lung_Cancer" "Bronchitis"
  .. ..$ nbr     : chr [1:2] "Lung_Cancer" "Bronchitis"
  .. ..$ parents : chr(0) 
  .. ..$ children: chr [1:2] "Lung_Cancer" "Bronchitis"
  ..$ Lung_Cancer         :List of 4
  .. ..$ mb      : chr [1:3] "Tuberculosis" "Smoke" "Either"
  .. ..$ nbr     : chr [1:2] "Smoke" "E

In [15]:
%%capture bnlearn_params_time
%time %R rfit = bn.fit(bn, data = states_df);

In [16]:
print(bnlearn_params_time)

CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 22.3 ms



In [17]:
%R str(rfit)

List of 8
 $ Visited_Asian_TB_HRA:List of 4
  ..$ node    : chr "Visited_Asian_TB_HRA"
  ..$ parents : chr(0) 
  ..$ children: chr "Tuberculosis"
  ..$ prob    : table [1:2(1d)] 0.009 0.991
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Tuberculosis        :List of 4
  ..$ node    : chr "Tuberculosis"
  ..$ parents : chr "Visited_Asian_TB_HRA"
  ..$ children: chr "Either"
  ..$ prob    : table [1:2, 1:2] 0 1 0.00605 0.99395
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ Tuberculosis        : chr [1:2] "0" "1"
  .. .. ..$ Visited_Asian_TB_HRA: chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Smoke               :List of 4
  ..$ node    : chr "Smoke"
  ..$ parents : chr(0) 
  ..$ children: chr [1:2] "Lung_Cancer" "Bronchitis"
  ..$ prob    : table [1:2(1d)] 0.499 0.501
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Lung_Cancer     

In [18]:
%%R
j = 0
for( nd in nodes(bn.fit)){
    j = j + 1
    cat('----------------', nd[1], '\n')
    cat('true:\n')
    print(coef(bn.fit)[j])
    cat('empirical:\n')
    print(coef(rfit)[j])
}

---------------- Visited_Asian_TB_HRA 
true:
$Visited_Asian_TB_HRA
 yes   no 
0.01 0.99 

empirical:
$Visited_Asian_TB_HRA
    0     1 
0.009 0.991 

---------------- Tuberculosis 
true:
$Tuberculosis
            Visited_Asian_TB_HRA
Tuberculosis  yes   no
         yes 0.05 0.01
         no  0.95 0.99

empirical:
$Tuberculosis
            Visited_Asian_TB_HRA
Tuberculosis          0          1
           0 0.00000000 0.00605449
           1 1.00000000 0.99394551

---------------- Smoke 
true:
$Smoke
yes  no 
0.5 0.5 

empirical:
$Smoke
    0     1 
0.499 0.501 

---------------- Lung_Cancer 
true:
$Lung_Cancer
           Smoke
Lung_Cancer  yes   no
        yes 0.10 0.01
        no  0.90 0.99

empirical:
$Lung_Cancer
           Smoke
Lung_Cancer          0          1
          0 0.11322645 0.01097804
          1 0.88677355 0.98902196

---------------- Bronchitis 
true:
$Bronchitis
          Smoke
Bronchitis yes  no
       yes 0.6 0.3
       no  0.4 0.7

empirical:
$Bronchitis
          

## Summary of Running Times

In [19]:
print("QFog:\n", qfog_params_time)
print("bnlearn:\n", bnlearn_params_time)

QFog:
 CPU times: user 56.6 ms, sys: 30 us, total: 56.6 ms
Wall time: 56.9 ms

bnlearn:
 CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 22.3 ms

